# AMSC698K Homework 6
##### Elijah Kin & Noorain Noorani

In [1]:
import math
import numpy as np
import qiskit
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
qiskit.__version__

'1.2.4'

### 1a - (60pts) Create a program for Shor's algorithm, i.e. for factorizing an integer N, including a quantum circuit to find the order r of a modulo $N (a^r := 1 mod N)$. 

First, create functions that perform the required processing steps (you may adapt the corresponding functions presented in class):

1) Create helper functions:

    (a) to check whether integer N is a perfect power ($N=p^q)$;

    (b) to check whether N and the base a (with $1<a<N$) are coprime;

    (c) to extract the exponent r and the factors of N.

2) Create quantum circuits:

- to perform QFT and inverse QFT on N qubits;
- to perform the double-controlled modular addition of a in the Fourier space and its inverse;
- to perform controlled modular multiplication by a 

3) Create a program that performs the factorization:

- get a positive odd integer N (user input) and check whether N is a perfect power (function 1a);
- get base a and check whether a and N are coprime (function 1b);
with n as the number of bits in N, create 3 quantum registers: a n qubit register (initialized to 1) and a n+2 auxiliary qubit register for performing modular multiplication, and a 2n qubit register to perform the inverse QFT (initialized to uniform superposition);
- apply the modular multiplication gate of power 2k controlled by qubit k (for each k=0,2n) in the 2n qubit register;
- apply the inverse QFT on the 2n qubit register and read it out (measure all 2n qubits);
- extract the exponent r and the factors of N from the measured data (function 1c)

### 1b - (40pts) Test your program on N=15,35,55, then submit the circuit first to "ionq_simulator" without and with noise model, check the results and then run it on an IonQ QPU.

### 1c - (extra credit 30pts) Finally, choose a number 156 < N < 256, which is the product of 2 primes, and factorize N using the IonQ Forte-1 QPU.